In [5]:
import torch
import torch.nn as nn
from torch.nn import init

net = nn.Sequential(nn.Linear(4,3),nn.ReLU(),nn.Linear(3,1))
print(net)
X = torch.rand(4,4)
y = net(X).sum()

Sequential(
  (0): Linear(in_features=4, out_features=3, bias=True)
  (1): ReLU()
  (2): Linear(in_features=3, out_features=1, bias=True)
)


# 访问模型参数

* Module.parameters() 
* Module.named_parameters()

In [7]:
print(type(net.named_parameters()))
for name,param in net.named_parameters():
    print(name,param.size())

<class 'generator'>
0.weight torch.Size([3, 4])
0.bias torch.Size([3])
2.weight torch.Size([1, 3])
2.bias torch.Size([1])


In [13]:
for name,param in net[0].named_parameters():
    print(name,param.size(),type(param))

weight torch.Size([3, 4]) <class 'torch.nn.parameter.Parameter'>
bias torch.Size([3]) <class 'torch.nn.parameter.Parameter'>


torch.nn.parameter.Parameter 是Tensor 的子类，它会自动加到模型的参数礼列表中。Tensor 不会

In [12]:
print(type(net.parameters()))
for i in net.parameters():
    print(i.size())

<class 'generator'>
torch.Size([3, 4])
torch.Size([3])
torch.Size([1, 3])
torch.Size([1])


In [17]:
class MyModule(nn.Module):
    def __init__(self,**kwargs):
        super(MyModule,self).__init__(**kwargs)
        self.weight1 = nn.parameter.Parameter(torch.randn(20,20))
        self.weight2 = torch.randn(20,20)
    def forward(self,x):
        pass
    
n =MyModule()
for name ,param in n.named_parameters():
    print(name,param.size())

weight1 torch.Size([20, 20])


In [18]:
weight_0 = list(net.parameters())[0]
print(weight_0.data)
print(weight_0.grad)
y.backward()
print(weight_0.grad)

tensor([[-0.3307, -0.4738, -0.2406, -0.3081],
        [-0.4691,  0.3086, -0.0662,  0.3466],
        [ 0.1486,  0.1304, -0.4879,  0.2416]])
None
tensor([[ 0.0000,  0.0000,  0.0000,  0.0000],
        [-0.1047, -0.3227, -0.2527, -0.2814],
        [ 0.0000,  0.0000,  0.0000,  0.0000]])


# 初始化模型参数 

* init 模块

In [20]:
for name,param in net.named_parameters():
    if "weight" in name:
        init.normal_(param,mean=0,std=0.01)
        print(name,param.data)

0.weight tensor([[-3.6712e-03,  4.2821e-04, -7.5942e-06,  5.7380e-03],
        [-8.2389e-05, -8.9720e-03,  1.5369e-03,  2.0711e-02],
        [-4.2912e-03,  5.4044e-03,  1.6350e-02,  2.0331e-02]])
2.weight tensor([[ 0.0081, -0.0135,  0.0037]])


In [24]:
for name,param in net.named_parameters():
    if "bias" in name:
        init.constant_(param,val=0)
        print(name,param.data)

0.bias tensor([0., 0., 0.])
2.bias tensor([0.])


## 自定义初始化

In [ ]:
def init_weight_(tensor):
    with torch.no_grad():
        tensor.uniform_(-10,10)
        tensor *= (tensor.abs()>=5).float()

# 共享模型参数

Sequentiald的模块是同一个Module实例时，参数是共享的

In [29]:
linear = nn.Linear(1,1,bias=False)
net = nn.Sequential(linear,linear)
print(net)
for name,param in net.named_parameters():
    init.constant_(param,val=3)
    print(name,param.data)

Sequential(
  (0): Linear(in_features=1, out_features=1, bias=False)
  (1): Linear(in_features=1, out_features=1, bias=False)
)
0.weight tensor([[3.]])


In [30]:
print(id(net[0])==id(net[1]))
print(id(net[0].weight)==id(net[1].weight))

True
True


In [31]:
x=torch.ones(1,1)
y=net(x).sum()
print(y)
y.backward()
print(net[0].weight.grad)

tensor(9., grad_fn=<SumBackward0>)
tensor([[6.]])


# 自定义层

** 需要训练的参数必须使用nn.Parameters类、ParameteList、ParameterDict